In [2]:
backtest_frame_path = 'E:/backtest-frame'
data_path = "E:/base_data"
import pandas as pd
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.rcParams['font.size'] = '20' # 设置字体大小 = '16' # 设置字体大小
sns.set()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import sys 
sys.path.append(backtest_frame_path)
import numba

from datetime import datetime
from tqdm import tqdm
from multiprocessing import Pool,cpu_count



In [3]:
from TDayProcessor import TDayProcessor
# TDayProcessor.data_path = data_path
from TimingFactor import TimingFactor
from BackTest import BackTest
from Data import Data
from TushareDownloader import TushareDownloader
from utils import *


%load_ext autoreload
%autoreload 2
%matplotlib inline

date_col = 'TradingDay'
secu_col = 'SecuCode'
key_col = [date_col,secu_col]

In [4]:
DB = Data(data_path,start_dt="2015-01-01",update=True)

0it [00:00, ?it/s]


0it [00:00, ?it/s]


100%|██████████| 5/5 [00:02<00:00,  1.68it/s]


100%|██████████| 5/5 [00:01<00:00,  2.67it/s]


In [5]:
base_df = DB.stock_basic_df
base_df.head()

,ts_code,SecuCode,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,1991-04-03
1,000002.SZ,000002,万科A,深圳,全国地产,主板,1991-01-29
2,000004.SZ,000004,国华网安,深圳,软件服务,主板,1991-01-14
3,000005.SZ,000005,ST星源,深圳,环境保护,主板,1990-12-10
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,1992-04-27


# 一、数据提取

## 1.1 特征生成

In [49]:
daily_df = DB.daily_df
daily_df = daily_df.merge(DB.ret_df[key_col + ['raw_close_close']],on=key_col,how='left')
daily_df['prec_open'] = (daily_df['open'] - daily_df['pre_close']) / daily_df['pre_close'] 
daily_df['open_low'] = (daily_df['low'] - daily_df['open']) / daily_df['open']
daily_df['low_high'] = (daily_df['high'] - daily_df['low']) / daily_df['low']
daily_df['high_close'] = (daily_df['high'] - daily_df['low']) / daily_df['low']
# daily_df['raw_close_close'] = 

feature_col = ['prec_open','open_low','low_high','high_close']
daily_df = daily_df[key_col + feature_col + ['raw_close_close']]
daily_df = daily_df.merge(base_df[[secu_col,'industry']], on=secu_col,how='left')
daily_df.dropna(inplace=True)
daily_df.head(3)


,TradingDay,SecuCode,prec_open,open_low,low_high,high_close,raw_close_close,industry
0,2015-01-05,000001,0.009470,-0.024390,0.043590,0.043590,0.011300,银行
1,2015-01-05,000002,0.035252,-0.011814,0.075246,0.075246,0.070143,全国地产
2,2015-01-05,000004,0.000642,-0.017972,0.039216,0.039216,0.007678,软件服务


## 1.2 训练集分割

In [50]:
all_industries = daily_df.industry.unique()
train_df = daily_df.query('TradingDay < "2022-01-01"')
valid_df = daily_df.query('TradingDay >= "2022-01-01" and TradingDay < "2023-01-01"')
test_df = daily_df.query('TradingDay >= "2023-01-01"')


## 1.3 抽样
从train_df 中抽样，对于时刻t，抽取前10日k线信息，共10*4 = 40维，用于聚类；后5日收益作为聚类后标签。


In [22]:
import random
import itertools

In [87]:
def sample(df,num_to_select=None,prev_nd=10,after_nd=5):
    """
    return: data,label
    """
    iterator = df.groupby('SecuCode')[feature_col+['raw_close_close']].rolling(prev_nd + after_nd)
    iterator_length = sum(1 for _ in iterator)
    if num_to_select is None:
        num_to_select = int(iterator_length/(prev_nd + after_nd))
    
    data = []
    label = []
    thresh = num_to_select / (len(selected_items) + 1)
    for item in tqdm(iterator,total = iterator_length):
        if (len(item) == prev_nd + after_nd) and (len(data) < num_to_select) and (random.random() < thresh):
            data.append(item[feature_col].values[0:prev_nd].reshape(1,-1)[0])
            label.append(item['raw_close_close'].values[prev_nd:].sum())
    
    return np.array(data), np.array(label)

In [70]:
# data,label = sample(train_df)

100%|██████████| 5486473/5486473 [19:03<00:00, 4799.42it/s]


In [74]:

# np.savez(os.path.join(data_path,'train_data.npz'), array1=data, array2=label)

# loaded_data = np.load(os.path.join(data_path,'train_data.npz'))
# loaded_array1 = loaded_data['array1']
# loaded_array2 = loaded_data['array2']

## 1.4 训练模型

In [125]:
from sklearn.decomposition import PCA,KernelPCA


In [118]:
for ind in  all_industries:
    ind = '电气设备'
    df = train_df.query('industry == @ind')

In [124]:
train_df['industry'].value_counts().head(10)

电气设备    309746
软件服务    277531
元器件     236383
化工原料    223466
汽车配件    210242
专用机械    190399
化学制药    154344
通信设备    147448
建筑工程    130563
医疗保健    130129
Name: industry, dtype: int64

In [119]:
data,label = sample(df,50000)


100%|██████████| 309746/309746 [00:11<00:00, 26500.96it/s] 


In [128]:
# 创建KernelPCA对象，指定核函数和降维后的维度
kernel_pca = KernelPCA(kernel='rbf', n_components=10)

# 对数据进行Kernel PCA转换
kernel_pca.fit(data)

# 输出降维后的数据
# print("Transformed Data (Kernel PCA):")
# print(transformed_data)


MemoryError: Unable to allocate 18.6 GiB for an array with shape (50000, 50000) and data type float64

In [121]:

# 创建PCA对象，指定降维后的维度
pca = PCA(n_components=10)

# 对数据进行PCA转换
pca.fit(data)

transformed_data = pca.fit_transform(data)


PCA(n_components=10)

In [122]:
len(data)

50000